In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_csv('/content/drive/My Drive/ML-Group2/tcd-ml-1920-group-income-train.csv')

test = pd.read_csv('/content/drive/My Drive/ML-Group2/tcd-ml-1920-group-income-test.csv')

rename_cols = {"Total Yearly Income [EUR]":'Income'}

train = train.rename(columns=rename_cols)

data = pd.concat([train,test],ignore_index=True)
data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].str.replace(' EUR','').astype('float')
year_median = data['Year of Record'].median()
data['Year of Record'].replace(np.nan, year_median, inplace=True)
data['Year of Record'].replace(np.nan, year_median, inplace=True)
data['Year of Record'] = data['Year of Record']**(1/2)
data['Satisfation with employer'].replace(np.nan, 'Missing')
age_median = data['Age'].median()
data['Age'].replace(np.nan, age_median, inplace=True)
data['Age'] = np.log((data['Age'])**(2))
data['Gender'].replace('0', 'Missing', inplace=True)
data['Gender'].replace('Other', 'Missing', inplace=True)
data['Gender'].replace('Unknown', 'Missing', inplace=True)
data['Gender'].replace(np.nan, 'Missing', inplace=True)
data['Hair Color'].replace('0', 'Missing', inplace=True)
data['Hair Color'].replace('Unknown', 'Missing', inplace=True)
data['Hair Color'].replace(np.nan, 'Missing', inplace=True)
data['Housing Situation'].replace('nA','Missing', inplace=True)
data['Housing Situation'].replace(0,'Missing', inplace=True)
data['Housing Situation'].replace('0','Missing', inplace=True)
data['University Degree'].replace('#N/A', 'No', inplace=True)
data['University Degree'].replace('0', 'No', inplace=True)
data['University Degree'].replace(np.nan, 'Missing', inplace=True)
data['Income'] = data['Income'].apply(np.log)
data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].apply(np.log)
data['Yearly Income in addition to Salary (e.g. Rental Income)'].replace(np.NINF, 0, inplace=True)
data['Size of City'] = data['Size of City'].apply(np.log)
# data['Housing Situation'].replace('Large Apartment','Large House', inplace=True)
# data['Housing Situation'].replace('Medium Apartment','Medium House', inplace=True)
# data['Housing Situation'].replace('Small Apartment','Small House', inplace=True)
# data['Housing Situation'].replace('0','Missing', inplace=True)
data['Work Experience in Current Job [years]'] = pd.to_numeric(data['Work Experience in Current Job [years]'],errors='coerce')


data.head(20)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Age,Body Height [cm],Country,Crime Level in the City of Employement,Gender,Hair Color,Housing Situation,Income,Instance,Profession,Satisfation with employer,Size of City,Total Yearly Income [EUR],University Degree,Wears Glasses,Work Experience in Current Job [years],Year of Record,Yearly Income in addition to Salary (e.g. Rental Income)
0,7.613325,182,Afghanistan,33,other,Black,Missing,8.729405,1,group head,Unhappy,10.133766,NaN,No,1,17.0,44.045431,0.0
1,5.666427,172,Afghanistan,25,female,Blond,Missing,8.827569,2,heavy vehicle and mobile equipment service tec...,Unhappy,14.638898,NaN,No,0,4.9,44.045431,0.0
2,7.742402,144,Afghanistan,34,female,Blond,Missing,9.066878,3,sorter,Unhappy,13.619659,NaN,Bachelor,0,21.0,44.045431,0.0
3,7.475339,152,Albania,70,female,Brown,Missing,7.783491,4,quality control senior engineer,Average,10.993345,NaN,No,1,18.0,44.045431,0.0
4,5.416100,180,Albania,51,other,Black,Missing,7.943137,5,logistician,Happy,10.064500,NaN,Master,1,8.0,44.045431,0.0
5,6.516193,212,Albania,61,male,Brown,Missing,7.852501,6,unix/linux systems lead,Average,10.329539,NaN,Bachelor,1,15.0,44.045431,0.0
6,6.182085,181,Albania,58,male,Black,Missing,8.112807,7,purchasing agent,Average,12.570792,NaN,Bachelor,1,12.0,44.045431,0.0
7,5.416100,161,Albania,51,female,Missing,Missing,8.210434,8,quality management specialist,Average,14.282584,NaN,Bachelor,0,6.3,44.045431,0.0
8,7.221836,168,Albania,68,male,Black,Missing,7.888473,9,investment officer,Happy,11.315864,NaN,Bachelor,1,15.0,44.045431,0.0
9,6.437752,186,Albania,60,Missing,Brown,Missing,8.268239,10,rigger,Average,14.540592,NaN,Bachelor,0,13.0,44.045431,0.0


In [0]:
train.shape

(1048574, 17)

In [0]:
test.shape

(369438, 17)

In [0]:
def create_cat_con(df,cats,cons,normalize=True):
    for i,cat in enumerate(cats):
        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict()
        nm = cat + '_FE_FULL'
        df[nm] = df[cat].map(vc)
        df[nm] = df[nm].astype('float32')
        for j,con in enumerate(cons):
#             print("cat %s con %s"%(cat,con))
            new_col = cat +'_'+ con
            print('timeblock frequency encoding:', new_col)
            df[new_col] = df[cat].astype(str)+'_'+df[con].astype(str)
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            df[new_col] = df[new_col].map(fq_encode)
            df[new_col] = df[new_col]/df[cat+'_FE_FULL']
    return df

cats = ['Gender','Age','Crime Level in the City of Employement',
        'Profession', 'University Degree','Satisfation with employer','Size of City']
cons = ['Housing Situation','Country','Year of Record','Yearly Income in addition to Salary (e.g. Rental Income)']

data = create_cat_con(data,cats,cons)

for col in train.dtypes[train.dtypes == 'object'].index.tolist():
    feat_le = LabelEncoder()
    feat_le.fit(data[col].unique().astype(str))
    data[col] = feat_le.transform(data[col].astype(str))

del_col = set(['Income','Instance', 'Total Yearly Income [EUR]'])
features_col =  list(set(data) - del_col)
features_col

timeblock frequency encoding: Gender_Housing Situation
timeblock frequency encoding: Gender_Country
timeblock frequency encoding: Gender_Year of Record
timeblock frequency encoding: Gender_Yearly Income in addition to Salary (e.g. Rental Income)
timeblock frequency encoding: Age_Housing Situation
timeblock frequency encoding: Age_Country
timeblock frequency encoding: Age_Year of Record
timeblock frequency encoding: Age_Yearly Income in addition to Salary (e.g. Rental Income)
timeblock frequency encoding: Crime Level in the City of Employement_Housing Situation
timeblock frequency encoding: Crime Level in the City of Employement_Country
timeblock frequency encoding: Crime Level in the City of Employement_Year of Record
timeblock frequency encoding: Crime Level in the City of Employement_Yearly Income in addition to Salary (e.g. Rental Income)
timeblock frequency encoding: Profession_Housing Situation
timeblock frequency encoding: Profession_Country
timeblock frequency encoding: Professi

['Age_FE_FULL',
 'Satisfation with employer_FE_FULL',
 'Profession_Country',
 'Crime Level in the City of Employement_Yearly Income in addition to Salary (e.g. Rental Income)',
 'Gender_Yearly Income in addition to Salary (e.g. Rental Income)',
 'Profession_Housing Situation',
 'Work Experience in Current Job [years]',
 'Satisfation with employer_Country',
 'Satisfation with employer_Yearly Income in addition to Salary (e.g. Rental Income)',
 'Size of City_Housing Situation',
 'University Degree_FE_FULL',
 'Gender',
 'Housing Situation',
 'Profession',
 'Yearly Income in addition to Salary (e.g. Rental Income)',
 'University Degree_Country',
 'Gender_FE_FULL',
 'Age_Yearly Income in addition to Salary (e.g. Rental Income)',
 'Profession_FE_FULL',
 'Age_Country',
 'Age_Housing Situation',
 'Country',
 'Wears Glasses',
 'Crime Level in the City of Employement_Housing Situation',
 'Gender_Housing Situation',
 'Size of City_FE_FULL',
 'Size of City_Year of Record',
 'Crime Level in the Cit

In [0]:
X_train = data[features_col].iloc[:1048574]
X_test = data[features_col].iloc[1048574:]

Y_train = data['Income'].iloc[:1048574]
X_test_id = data['Instance'].iloc[1048574:]
#x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=1234)

#test_data = test
#test_data = test_data.drop('Total Yearly Income [EUR]', 1)
X_train.head(20)


,Age_FE_FULL,Satisfation with employer_FE_FULL,Profession_Country,Crime Level in the City of Employement_Yearly Income in addition to Salary (e.g. Rental Income),Gender_Yearly Income in addition to Salary (e.g. Rental Income),Profession_Housing Situation,Work Experience in Current Job [years],Satisfation with employer_Country,Satisfation with employer_Yearly Income in addition to Salary (e.g. Rental Income),Size of City_Housing Situation,University Degree_FE_FULL,Gender,Housing Situation,Profession,Yearly Income in addition to Salary (e.g. Rental Income),University Degree_Country,Gender_FE_FULL,Age_Yearly Income in addition to Salary (e.g. Rental Income),Profession_FE_FULL,Age_Country,Age_Housing Situation,Country,Wears Glasses,Crime Level in the City of Employement_Housing Situation,Gender_Housing Situation,Size of City_FE_FULL,Size of City_Year of Record,Crime Level in the City of Employement_Country,Gender_Country,University Degree_Yearly Income in addition to Salary (e.g. Rental Income),University Degree_Year of Record,Satisfation with employer_Housing Situation,Age,Body Height [cm],Age_Year of Record,Size of City,Hair Color,University Degree_Housing Situation,Crime Level in the City of Employement,Crime Level in the City of Employement_Year of Record,Profession_Year of Record,Profession_Yearly Income in addition to Salary (e.g. Rental Income),Size of City_Yearly Income in addition to Salary (e.g. Rental Income),Crime Level in the City of Employement_FE_FULL,Size of City_Country,Year of Record,University Degree,Gender_Year of Record,Satisfation with employer,Satisfation with employer_Year of Record
0,0.018091,0.015185,0.001271,0.901018,0.901823,0.238882,10,0.007198,0.901314,0.250000,0.245693,4,5,610,0,0.000132,0.240295,0.905001,0.000555,0.000234,0.251394,1,1,0.255341,0.256661,2.820851e-06,0.250000,0.002461,0.000179,0.901379,0.011694,0.251474,7.613325,182,0.012630,10.133766,0,0.258279,33,0.013757,0.012706,0.913596,1.000000,0.006305,0.250000,44.045431,3,0.011707,3,0.012121
1,0.023940,0.015185,0.001214,0.899627,0.902832,0.251214,40,0.007198,0.901314,1.000000,0.245693,2,5,622,0,0.000132,0.226179,0.897517,0.000581,0.000118,0.263676,1,0,0.248533,0.274145,7.052126e-07,1.000000,0.001600,0.000109,0.901379,0.011694,0.251474,5.666427,172,0.012520,14.638898,1,0.258279,25,0.010240,0.013350,0.901699,1.000000,0.006611,1.000000,44.045431,3,0.012961,3,0.012121
2,0.016077,0.015185,0.000833,0.903662,0.902832,0.284167,17,0.007198,0.901314,0.500000,0.378217,2,5,1194,0,0.000155,0.226179,0.901478,0.000846,0.000175,0.255735,1,0,0.255125,0.274145,1.410425e-06,0.500000,0.002725,0.000109,0.901105,0.011730,0.251474,7.742402,144,0.011230,13.619659,1,0.257889,34,0.012205,0.007500,0.904167,1.000000,0.005951,0.500000,44.045431,0,0.012961,3,0.012121
3,0.019548,0.464753,0.003873,0.901139,0.902832,0.251356,11,0.003478,0.902268,0.333333,0.245693,2,5,1008,0,0.003436,0.226179,0.899527,0.001821,0.003211,0.258018,2,1,0.245144,0.274145,4.231276e-06,0.166667,0.024916,0.003445,0.901379,0.011694,0.256890,7.475339,152,0.011364,10.993345,2,0.258279,70,0.012190,0.013168,0.904725,1.000000,0.005264,0.166667,44.045431,3,0.012961,0,0.011852
4,0.024421,0.335391,0.003605,0.902547,0.901823,0.255922,54,0.003556,0.901076,0.125000,0.239070,4,5,760,0,0.003466,0.240295,0.901037,0.001370,0.003350,0.256115,2,1,0.262604,0.256661,5.641701e-06,0.125000,0.015073,0.003328,0.902482,0.012439,0.258099,5.416100,180,0.013197,10.064500,0,0.257336,51,0.013514,0.009784,0.897013,0.625000,0.005427,0.125000,44.045431,1,0.011707,1,0.011737
5,0.023968,0.464753,0.004751,0.905016,0.900842,0.237530,8,0.003478,0.902268,0.166667,0.378217,3,5,1319,0,0.003468,0.381580,0.905464,0.000297,0.003237,0.260747,2,1,0.256563,0.257017,4.231276e-06,0.166667,0.011740,0.003450,0.901105,0.011730,0.256890,6.516193,212,0.012770,10.329539,2,0.257889,61,0.011419,0.011876,0.900238,0.833333,0.006608,0.166667,44.045431,0,0.011793,0,0.011852
6,0.024336,0.464753,0.005660,0.903982,0.900842,0.250189,5,0.003478,0.902268,0.500000,0.3

In [0]:
params = {
          'max_depth': 22,
          'learning_rate': 0.01,
          "boosting": "gbdt",
          "bagging_seed": 11,
          "metric": 'mse',
          "verbosity": -1
         }
trn_data = lgb.Dataset(X_train, label=Y_train)
#val_data = lgb.Dataset(x_val, label=y_val)
# test_data = lgb.Dataset(X_test)
clf = lgb.train(params, trn_data, 100000)
'done training'
#pre_test_lgb = clf.predict(X_test)
#dataframe = pd.DataFrame(data=pre_test_lgb)
#'done predicting'
#dataframe = np.exp(dataframe)
#'done de-logging'
#dataframe.head(20)


'done training'

In [0]:
pre_val_lgb = clf.predict(X_test)
dataframe_predictions = pd.Series(pre_val_lgb)
'done predicting'
dataframe_predictions = np.exp(dataframe_predictions)
'done de-logging'

'done de-logging'

In [0]:
#from sklearn import metrics
#import matplotlib.pyplot as plt

#def calculate_metrics():
#    df = pd.DataFrame({'Actual': y_val, 'Predicted': dataframe_predictions})
#    df1 = df.head(25)
#    # df1.plot(kind='bar', figsize=(10, 8))
#    # plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
#    # plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
#    # plt.show()
#    print('Mean Absolute Error:', metrics.mean_absolute_error(y_val, dataframe_predictions))
#    print('Mean Squared Error:', metrics.mean_squared_error(y_val, dataframe_predictions))
#    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, dataframe_predictions)))

#calculate_metrics()

In [0]:
instances = np.arange((dataframe_predictions.shape[0])+1)
instances = instances[1:]
instances
sub_df = pd.DataFrame({'Instance':instances,
                       'Total Yearly Income [EUR]':dataframe_predictions})
sub_df.head()

,Instance,Total Yearly Income [EUR]
0,1,33000.618567
1,2,6574.592840
2,3,3340.637022
3,4,75092.937733
4,5,3901.874968


In [0]:
sub_df.to_csv("submission1.csv",index=False)
!cp submission1.csv drive/My\ Drive/
'done'

'done'

In [0]:
sub_df.shape

(369438, 2)